# Exploratory data analysis

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from colorama import Fore, Style
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import sys
import os

## 1. Loading the Data
- Importing the preprocessed data


In [2]:
data_path = os.path.join('..','raw_data', 'hotel_bookings_raw.csv')

df = pd.read_csv(data_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 43 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [3]:
# hotel -> 0 und 1
# is_canceled, is_repeated_guest -> keep like this
# stays_in_weekend_nights, stays_in_week_nights, lead_time, adults, adr, CPI_AVG, INFLATION, INFLATION_CHG, CSMR_SENT, UNRATE, INTRSRT, GDP, FUEL_PRCS, CPI_HOTELS, US_GINI, DIS_INC-> scale
# arrival_date_month -> change to numbers
# arrival_date_year, arrival_date_week_number, arrival_date_day_of_month, children, babies, previous_cancellations, previous_bookings_not_canceled, reserved_room_type, assigned_room_type, booking_changes, deposit_type, agent, days_in_waiting_list, customer_type, required_car_parking_spaces, total_of_special_requests, reservation_status_date, MO_YR, -> drop
# meal, country, market_segment, distribution_channel, reservation_status -> categorical (3-5 categories, encoden, maybe drop undefined)

In [4]:
#Drop columns
df = df.drop(columns=['arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'children', 'babies', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status_date', 'MO_YR'])

In [5]:
#Drop duplicates
df.duplicated().sum()
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83123 entries, 0 to 119389
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hotel                    83123 non-null  object 
 1   is_canceled              83123 non-null  int64  
 2   lead_time                83123 non-null  int64  
 3   arrival_date_month       83123 non-null  object 
 4   stays_in_weekend_nights  83123 non-null  int64  
 5   stays_in_week_nights     83123 non-null  int64  
 6   adults                   83123 non-null  int64  
 7   meal                     83123 non-null  object 
 8   country                  82685 non-null  object 
 9   market_segment           83123 non-null  object 
 10  distribution_channel     83123 non-null  object 
 11  is_repeated_guest        83123 non-null  int64  
 12  adr                      83123 non-null  float64
 13  reservation_status       83123 non-null  object 
 14  CPI_AVG                  8

In [7]:
#Drop duplicates
df.dropna(inplace=True)
df.isna().sum()

hotel                      0
is_canceled                0
lead_time                  0
arrival_date_month         0
stays_in_weekend_nights    0
stays_in_week_nights       0
adults                     0
meal                       0
country                    0
market_segment             0
distribution_channel       0
is_repeated_guest          0
adr                        0
reservation_status         0
CPI_AVG                    0
INFLATION                  0
INFLATION_CHG              0
CSMR_SENT                  0
UNRATE                     0
INTRSRT                    0
GDP                        0
FUEL_PRCS                  0
CPI_HOTELS                 0
US_GINI                    0
DIS_INC                    0
dtype: int64

In [8]:
df['arrival_date_month']

0           July
1           July
2           July
3           July
4           July
           ...  
119385    August
119386    August
119387    August
119388    August
119389    August
Name: arrival_date_month, Length: 82675, dtype: object

In [9]:
#Change months to number
# Create a mapping of month names to numbers
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}
#Strip spaces
df['arrival_date_month'] = df['arrival_date_month'].str.strip()
# Replace month names with numbers
df['arrival_date_month'] = df['arrival_date_month'].map(month_mapping)
df['arrival_date_month']


0         7
1         7
2         7
3         7
4         7
         ..
119385    8
119386    8
119387    8
119388    8
119389    8
Name: arrival_date_month, Length: 82675, dtype: int64

In [10]:
#Change hotel to binary
hotel_mapping = {
    'City Hotel': 1,
    'Resort Hotel': 0
}
#Strip spaces
df['hotel'] = df['hotel'].str.strip()
# Replace month names with numbers
df['hotel'] = df['hotel'].map(hotel_mapping)
df['hotel']


0         0
1         0
2         0
3         0
4         0
         ..
119385    1
119386    1
119387    1
119388    1
119389    1
Name: hotel, Length: 82675, dtype: int64

In [13]:
# List of columns to check unique values
columns_to_check = ["meal", "country", "market_segment", "distribution_channel", "reservation_status"]

# Iterate through each column and print unique values
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)

Unique values in column 'meal':
['BB' 'FB' 'HB' 'SC' 'Undefined']
Unique values in column 'country':
['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG' 'POL' 'DEU'
 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST' 'CZE'
 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR' 'UKR'
 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO' 'ISR'
 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM' 'HRV'
 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY' 'KWT'
 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN' 'SYC'
 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB' 'CMR'
 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI' 'SAU'
 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB' 'NPL'
 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KNA' 'ETH' 'IRQ' 'HND' 'RWA' 'KHM'
 'MCO' 'BGD' 'IMN' 'TJK' 'NIC' 'BEN' 'VGB' 'TZA' 'GAB' 'GHA' 'TMP' 'GLP'
 'KEN' 'LIE' 'GNB' 'MNE'

In [16]:
#Drop undefined

df = df[df["meal"] != "Undefined"]
df = df[df["market_segment"] != "Undefined"]
df = df[df["distribution_channel"] != "Undefined"]


In [17]:
# List of columns to check unique values
columns_to_check = ["meal", "country", "market_segment", "distribution_channel", "reservation_status"]

# Iterate through each column and print unique values
for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)

Unique values in column 'meal':
['BB' 'FB' 'HB' 'SC']
Unique values in column 'country':
['PRT' 'GBR' 'USA' 'ESP' 'IRL' 'FRA' 'ROU' 'NOR' 'OMN' 'ARG' 'POL' 'DEU'
 'BEL' 'CHE' 'CN' 'GRC' 'ITA' 'NLD' 'DNK' 'RUS' 'SWE' 'AUS' 'EST' 'CZE'
 'BRA' 'FIN' 'MOZ' 'BWA' 'LUX' 'SVN' 'ALB' 'IND' 'CHN' 'MEX' 'MAR' 'UKR'
 'SMR' 'LVA' 'PRI' 'SRB' 'CHL' 'AUT' 'BLR' 'LTU' 'TUR' 'ZAF' 'AGO' 'ISR'
 'CYM' 'ZMB' 'CPV' 'ZWE' 'DZA' 'KOR' 'CRI' 'HUN' 'ARE' 'TUN' 'JAM' 'HRV'
 'HKG' 'IRN' 'GEO' 'AND' 'GIB' 'URY' 'JEY' 'CAF' 'CYP' 'COL' 'GGY' 'KWT'
 'NGA' 'MDV' 'VEN' 'SVK' 'FJI' 'KAZ' 'PAK' 'IDN' 'LBN' 'PHL' 'SEN' 'SYC'
 'AZE' 'BHR' 'NZL' 'THA' 'DOM' 'MKD' 'MYS' 'ARM' 'JPN' 'LKA' 'CUB' 'CMR'
 'BIH' 'MUS' 'COM' 'SUR' 'UGA' 'BGR' 'CIV' 'JOR' 'SYR' 'SGP' 'BDI' 'SAU'
 'VNM' 'PLW' 'QAT' 'EGY' 'PER' 'MLT' 'MWI' 'ECU' 'MDG' 'ISL' 'UZB' 'NPL'
 'BHS' 'MAC' 'TGO' 'TWN' 'DJI' 'STP' 'KNA' 'ETH' 'IRQ' 'HND' 'RWA' 'KHM'
 'MCO' 'BGD' 'IMN' 'TJK' 'NIC' 'BEN' 'VGB' 'TZA' 'GAB' 'GHA' 'TMP' 'GLP'
 'KEN' 'LIE' 'GNB' 'MNE' 'UMI' 'MYT'

In [21]:
# Encode country
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['country'] = label_encoder.fit_transform(df['country'])

# Display the unique values in the encoded target column
print("Unique values in 'country':", df['country'].unique())

# Display the mapping of original country codes to encoded labels
country_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Country mapping:\n", country_mapping)

Unique values in 'country': [135  59 169  51  76  56 139 124 127   6 133  43  15  29  34  66  81 123
  46 140 154  11  52  42  25  54 113  27 100 153   3  75  31 107 103 166
 148 101 134 149  30  12  23  99 162 174   1  80  40 175  37 176  48  91
  38  72   5 161  82  71  69  77  60   4  63 168  83  28  41  35  61  92
 121 106 171 152  55  86 128  73  94 131 144 155  13  20 126 158  47 108
 117   7  85  98  39  33  22 115  36 151 165  19  32  84 156 145  14 142
 173 132 138  50 130 110 116  49 105  79 170 125  21 102 157 163  44 150
  90  53  78  70 141  88 104  18  74 159 122  16 172 164  58  62 160  64
  87  97  65 112 167 118  57 111 129  17  95 109 119  24 136  26   0   2
 147  45 137  68  96   9  67   8 114 120  89 143  10 146  93]
Country mapping:
 {'ABW': 0, 'AGO': 1, 'AIA': 2, 'ALB': 3, 'AND': 4, 'ARE': 5, 'ARG': 6, 'ARM': 7, 'ASM': 8, 'ATA': 9, 'ATF': 10, 'AUS': 11, 'AUT': 12, 'AZE': 13, 'BDI': 14, 'BEL': 15, 'BEN': 16, 'BFA': 17, 'BGD': 18, 'BGR': 19, 'BHR': 20, 'BHS': 21, 'B

In [23]:
# One Hot Encode meal, country, market_segment, distribution_channel, reservation_status -> categorical (3-5 categories, encoden)
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Columns to one-hot encode
columns_to_encode = ['meal', 'market_segment', 'distribution_channel', 'reservation_status']

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' avoids the dummy variable trap

# Fit and transform the data
encoded_data = ohe.fit_transform(df[columns_to_encode])

# Convert encoded data to DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(columns_to_encode))

# Concatenate the encoded columns with the original DataFrame
df_encoded = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Drop the original columns that were encoded
df_encoded.drop(columns=columns_to_encode, inplace=True)

# Display the first few rows of the updated DataFrame
print(df_encoded.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82269 entries, 0 to 82268
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   hotel                         82269 non-null  int64  
 1   is_canceled                   82269 non-null  int64  
 2   lead_time                     82269 non-null  int64  
 3   arrival_date_month            82269 non-null  int64  
 4   stays_in_weekend_nights       82269 non-null  int64  
 5   stays_in_week_nights          82269 non-null  int64  
 6   adults                        82269 non-null  int64  
 7   country                       82269 non-null  int64  
 8   is_repeated_guest             82269 non-null  int64  
 9   adr                           82269 non-null  float64
 10  CPI_AVG                       82269 non-null  float64
 11  INFLATION                     82269 non-null  float64
 12  INFLATION_CHG                 82269 non-null  float64
 13  C

In [ ]:
#Scale numerical features: country, stays_in_weekend_nights, stays_in_week_nights, lead_time, adults, adr, CPI_AVG, INFLATION, INFLATION_CHG, CSMR_SENT, UNRATE, INTRSRT, GDP, FUEL_PRCS, CPI_HOTELS, US_GINI, DIS_INC
#MinMaxScaler:
#StandardScaler:


In [7]:
#(from ml_logic.preprocessor import preprocess_features)

# Import the preprocessing function
sys.path.append('..')  # Adjust the path to your file structure
from demand_predictor.ml_logic.preprocessor import preprocess_features  # Adjust the import according to your file structure

# Define the path to your raw data CSV file
data_path = '../raw_data/hotel_bookings_raw.csv'  # Adjust the path if necessary

# Load the data
df = pd.read_csv(data_path)

# Display the first few rows of the dataframe
print(df.head())

# Preprocess the data
X_processed = preprocess_features(df)

# Display the shape of the processed data
print("Processed data shape:", X_processed.shape)

          hotel  is_canceled  lead_time  arrival_date_year arrival_date_month  \
0  Resort Hotel            0        342               2015               July   
1  Resort Hotel            0        737               2015               July   
2  Resort Hotel            0          7               2015               July   
3  Resort Hotel            0         13               2015               July   
4  Resort Hotel            0         14               2015               July   

   arrival_date_week_number  arrival_date_day_of_month  \
0                        27                          1   
1                        27                          1   
2                        27                          1   
3                        27                          1   
4                        27                          1   

   stays_in_weekend_nights  stays_in_week_nights  adults  ...  INFLATION  \
0                        0                     0       2  ...        1.8   
1           

## 2. Data Visualization
- Univariate analysis
- Bivariate analysis (scatterplot)
- Multivariate analysis
- Autogluon summary
- Histograms and Density PLots
- Boxplots
- Heatmaps
- Bar Plots
- Pair Plots
- Correlation Matrix
- Pairwise Correlation
- etc.


In [ ]:
# #Summarize the dataset using AutoGluon
# from autogluon.tabular import TabularDataset, TabularPredictor
# ag_data = TabularDataset(data_path)
# print(ag_data.describe())

## 3. Feature Engineering
- Creating new features
- Transforming existing features

## 4. Summary and Next Steps
- Key insights from the analysis
- Potential next steps for modeling